In [1]:
# allows to import of modules
import os
import sys
nb_dir = os.path.split(os.getcwd())[0]
nb_dir
if nb_dir not in sys.path:
    sys.path.append(nb_dir)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
## BADD-- supressing f1 score ill-defined and being set to 0.0 in labels with no true samples warning
import warnings
warnings.filterwarnings("ignore")

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np

In [5]:
import  datatset
import utils
from torch.utils.data import DataLoader

In [6]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")
# TRAIN_PATH = Path("/home/ubuntu/data-1/nybg2020/train/")

In [28]:
list(TRAIN_PATH.iterdir())

[PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/models'),
 PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/validation.csv'),
 PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/images'),
 PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/train.csv'),
 PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/metadata.json'),
 PosixPath('/home/shaun/personal/kaggle-data/nybg2020/train/logs')]

# Data

In [ ]:
df = pd.read_csv(TRAIN_PATH/"train.csv")
df_train = df[[ "file_name", "category_id"]]

In [ ]:
def over_sample_class(x):
    if len(x) < 20:
        return x.sample(n=20, replace=True) 
    return x

In [ ]:
def sample_class(x):
    if len(x) > 100:
        return x

In [ ]:
classes_map = utils.load_json(TRAIN_PATH/"classes-map.json")

In [ ]:
counts  = df_train["category_id"].value_counts()

In [ ]:
len(df_train), len(df_train["category_id"].value_counts())

### Over sample and apply data augmentation

In [ ]:
# Over Sample Class
new_df = df_train.groupby('category_id').apply(over_sample_class).reset_index(drop = True)

In [ ]:
# apply data augmentation to labels with less then 21 exampels
labels_aug = counts.keys()[counts < 21] # Get labels that have less then 21 exmaples
augment = new_df["category_id"].isin(labels_aug) # Get list of items to apply aug

In [ ]:
new_df["augment"] = augment #np.zeros(len(new_df)) == 1 # set all augemntaion to zero

In [ ]:
np.sum(new_df.augment)

### Update Labels

In [ ]:
# Update class labels
new_labels = [classes_map[str(i)] for i in new_df["category_id"]]
new_df["new_labels"] = new_labels

In [ ]:
num_calsses = len(counts)

In [ ]:
num_calsses

In [ ]:
from tqdm import tqdm_notebook

# there is a better way to do this
def sample_for_validation(df, n=4):   
    validation_set = {}
    bool_arr = []
    for i,row in tqdm_notebook(df.iterrows(), total=df.shape[0]):
        if validation_set.get(row["new_labels"], 0) == n:
            bool_arr.append(False) 
        else:
            validation_set[row["new_labels"]] = validation_set.get(row["new_labels"], 0) + 1
            bool_arr.append(True) 

        
    return bool_arr

### Split into train and validation

In [ ]:
new_df["is_valid"] = sample_for_validation(new_df, n=4)

In [ ]:
train_df, valid_df = new_df[~new_df["is_valid"]] , new_df[new_df["is_valid"]]

In [ ]:
# len(train_df), len(valid_df)
# (703500, 14766)

In [ ]:
len(train_df), len(valid_df)

In [ ]:
valid_df.to_csv(TRAIN_PATH/"validation.csv")

In [ ]:
train_df.to_csv(TRAIN_PATH/"train-9.csv")

# RESTART HERE

In [15]:
train_df = pd.read_csv(TRAIN_PATH/"train-9.csv")

In [16]:
num_calsses = len(train_df["category_id"].value_counts())

In [17]:
ds = datatset.HerbDataSet(train_df, TRAIN_PATH, 256, label="new_labels",device="gpu")

In [18]:
data_loader = DataLoader(ds, batch_size=8, shuffle=True)

# Model

In [11]:
import torch
import pretrainedmodels
import model

In [19]:
m = pretrainedmodels.se_resnext50_32x4d(num_classes=num_classes, pretrained='imagenet')
# in_features = m.last_linear.in_features
# m.last_linear = model.get_head(in_features)


AssertionError: num_classes should be 1000, but is 32093

In [ ]:
import torch.nn as nn
import torch

In [13]:
assert m.last_linear[-1].out_features == num_calsses

NameError: name 'm' is not defined

# Tracking

In [17]:
from torch.utils.tensorboard import SummaryWriter
import torch
import torch.nn as nn

### RUN

In [18]:
RUN = "run-11"

In [19]:
# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(TRAIN_PATH/f'logs/{RUN}/train/')

In [20]:
model_path=TRAIN_PATH/f'models-{RUN}/'

In [21]:
if not model_path.exists():
    model_path.mkdir()

In [22]:
import metrics
import train

In [23]:
acc =  metrics.Accuracy()
f1 = metrics.F1()
manager = metrics.MetricManager([acc, f1], writer)

# Train

In [24]:
import focal_loss

In [25]:
criterion =  focal_loss.FocalLoss()

In [26]:
layer_lrs = [
#     {'params': m.layer1.parameters(), 'lr': 1e-6},
#     {'params': m.layer2.parameters(), 'lr': 1e-6},
#     {'params': m.layer3.parameters(), 'lr': 1e-5},
#     {'params': m.layer4.parameters(), 'lr': 1e-4},
    {'params': m.last_linear.parameters(), 'lr': 1e-3}
]

In [27]:
optimizer = torch.optim.Adam(layer_lrs, lr=1e-3)

In [28]:
trainer = train.Training(manager, criterion, optimizer, data_loader, 5, m, model_path)

In [29]:
! export CUDA_LAUNCH_BLOCKING=1

In [30]:
trainer.run()

Epoch 0/5


RuntimeError: size mismatch, m1: [8 x 8192], m2: [2048 x 4096] at /pytorch/aten/src/THC/generic/THCTensorMathBlas.cu:268